In [3]:
!pip install tensorflow
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np 
import regex as re

In [4]:
import re
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def file_to_sentence_list(file_path):
    # Download the content if it's a URL
    response = requests.get(file_path)
    text = response.text

    # Splitting the text into sentences using delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

# Use the URL to download the text
book_url = "https://www.gutenberg.org/files/1342/1342-0.txt"
text_data = file_to_sentence_list(book_url)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (126981, 154)
Shape of y: (126981, 8709)


In [5]:
# Define the model 
model = Sequential() 
model.add(Embedding(total_words, 10, 
					input_length=max_sequence_len-1)) 
model.add(LSTM(128)) 
model.add(Dense(total_words, activation='softmax')) 
model.compile(loss='categorical_crossentropy', 
			optimizer='adam', metrics=['accuracy']) 


c:\Users\Bhindu Suma\anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
# Train the model 
model.fit(X, y, epochs=5, verbose=1) 


Epoch 1/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 311s 78ms/step - accuracy: 0.1129 - loss: 5.6248
Epoch 2/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 361s 91ms/step - accuracy: 0.1345 - loss: 5.2816
Epoch 3/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 290s 73ms/step - accuracy: 0.1461 - loss: 5.0346
Epoch 4/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 421s 106ms/step - accuracy: 0.1556 - loss: 4.8152
Epoch 5/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 295s 74ms/step - accuracy: 0.1680 - loss: 4.6119


In [6]:
# Generate next word predictions 
seed_text = "its good to be"
next_words = 10

for _ in range(next_words): 
	token_list = tokenizer.texts_to_sequences([seed_text])[0] 
	token_list = pad_sequences( 
		[token_list], maxlen=max_sequence_len-1, padding='pre') 
	predicted_probs = model.predict(token_list) 
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)] 
	seed_text += " " + predicted_word 

print("Next predicted words:", seed_text) 


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 767ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Next predicted words: its good to be message message abiding message abiding vex message attractions indignation 290


In [8]:


# Train the model
model.fit(X, y, epochs=5, verbose=1)

# Save the model
model.save('text_generation_model.h5')



Epoch 1/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 344s 85ms/step - accuracy: 0.0449 - loss: 6.7500
Epoch 2/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 316s 80ms/step - accuracy: 0.0923 - loss: 5.9179
Epoch 3/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 318s 80ms/step - accuracy: 0.1211 - loss: 5.4848
Epoch 4/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 305s 77ms/step - accuracy: 0.1398 - loss: 5.1961
Epoch 5/5
3969/3969 ━━━━━━━━━━━━━━━━━━━━ 310s 78ms/step - accuracy: 0.1515 - loss: 4.9461
